Dr Oliviero Andreussi, olivieroandreuss@boisestate.edu

Boise State University, Department of Chemistry and Biochemistry

# Data Analysis: Plots, Errors, Fits

Before we start, let us import some of the main modules that we will need for this lecture. These modules have already been introduced in the previous lecture. However, in the following we will introduce some new modules, we will add more details about them in the right sections.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp
#from google.colab import drive
#drive.mount('/content/gdrive')

You should now specify the local path to the folder containing your data files. Remember to put a '/' at the end of the path and double check that the path looks right

In [ ]:
base_path = './' # '/content/gdrive/MyDrive/' # this is the default path of your google drive
my_path = 'Test_Files/' # make sure you change this to the correct path of the folder with the files
path = base_path + my_path

## How to... Make a Plot

Step 1: Load your data

In [ ]:
file = 'uv-vis.csv'
data = pd.read_csv(path+file)

Step 2: Check that the data was read correctly. This usually involve looking at the head, tail, and info.

In [ ]:
print(data.head())
print(data.tail())
print(data.info())

Step 3: Plot using matplotlib. Remember to add axes labels using `plt.xlabel` and `plt.ylabel`. Remember to add the units. 

In [ ]:
plt.plot(data['Time'],data['Absorbance'])
plt.plot(data['Time2'],data['Absorbance2'])
plt.ylabel('Absorbance (a.u.)')
plt.xlabel('Time (s)')
plt.show() # in a notebook this is not essential, but it is cleaner

You can specify a range for x or y (zoom in or out) using `plt.xlim` and `plt.ylim`.

In [ ]:
plt.plot(data['Time'],data['Absorbance'])
plt.xlim(1,4) # from 1 to 4 seconds
plt.ylim(0.,0.3) # correspondent zoom on the y axis
plt.ylabel('Absorbance (a.u.)')
plt.xlabel('Time (s)')
plt.show() # in a notebook this is not essential, but it is cleaner

if you want to use a log scale for the vertical axis, add a legend, add markers to the lines, specify the color of the curve and the size of the marker, see below

In [ ]:
plt.semilogy(data['Time'],data['Absorbance'],'o-',ms=1,color='orange',label='Set A') # semilogy uses a log scale for the y axis
#'o-' means a round marker and a continuous line 
# ms controls the marker size
# color is pretty intuitive
# label adds a text to the curve
plt.ylabel('Absorbance (a.u.)')
plt.xlabel('Time (s)')
plt.legend() # this will add a legend with all the specified labels
plt.show() # in a notebook this is not essential, but it is cleaner

## How to... Have Multiple Plots in the Same Figure

Perform Step 1 to 3 from above and make sure the individual plots are what you want.

In [ ]:
file = 'uv-vis.csv'
data = pd.read_csv(path+file)

We can create multiple plots using the `plt.subplot(n,m,i)`. The command expects three integers, which specify how many rows (`m`) and columns (`n`) of subplots we want, as well as which subplot (`i`) we want to work on (ranging from 1 to m*n).

In [ ]:
plt.figure(figsize=(6,6)) # if you have multiple plots you may want to change the overall dimensions of the figure
plt.subplot(2,1,1) # 2 rows and 1 column of plots, working on the first one (top)
plt.semilogy(data['Time'],data['Absorbance'], label='Absorbance') # semilogy uses a log scale for the y axis
plt.legend()
plt.subplot(2,1,2) # 2 rows and 1 column of plots, working on the second plot (bottom)
plt.plot(data['Time'],data['Absorbance'])
plt.xlabel('Time (s)')
plt.show()

## How to... Report Measurement Estimates and Confidence Intervals for Large Samples

Multiple sources of errors affect our measurements. If these sources of errors are small and random, and in the limit of an infinite number of measurements, the results will end up being distributed according to a bell curve (a.k.a. a Gaussian or a normal distribution). The exact value will correspond to the center of the Gaussian, usually indicated by the parameter $\mu$. The Central Limit Theorem tells us that the best estimate for our quantity is represented by the mean of the repeated measures, i.e.,  $m=\frac{1}{N}\sum x_i$. 

Step 1: Load your data

In [ ]:
file = 'ages.csv'
ages = pd.read_csv(path+file,names=['age'])

Step 2: Check your data, print it, look at its statistical descriptors, plot an histogram

In [ ]:
print(ages.head())
print(ages.describe())

In [ ]:
plt.hist(ages,bins=20)
plt.show()

Step 3: Compute the mean and standard deviation of your sample

The sample of measurements is not really symmetric or looking like a Gaussian, that's because we only have a finite number $N$ of measurements. This means that using the sample's mean $m$ as our estimate for the center of the Gaussian $\mu$ comes with an error. The Central Limit Theorem tells us that the Standard Error of the Mean (SEM, or $\sigma_m$) is given by $\sigma_m$=$\sigma/\sqrt{N}$, where $\sigma$ is the spread of our Gaussian distribution. Like for the center of the distribution, we can estimate this spread from the sample's data, $s^2=\frac{1}{N-1}\sum (x_i-m)^2$. Since we already used the data to estimate $\mu$, we have one less degree of freedom at the denominator (in practice this tells us that we cannot compute the spread if we have only one datapoint).

The `describe()` method provides immediate access to both the mean and the standard deviation of the sample. We can also compute them using methods of numpy arrays as follows

In [ ]:
mean_sample = ages.values.mean()
std_sample = ages.values.std(ddof=1) # ddof, or delta degree of freedom, is needed to have the -1 at the denominator
N_sample = len(ages.values)
print(mean_sample,std_sample,N_sample)

Step 4: Compute the standard deviation of the mean from the numbers above or the ones returned by `describe()`

In [ ]:
SEM = std_sample/np.sqrt(N_sample)
print(SEM)

but note that there is also a dedicated function in the SciPy module

In [ ]:
from scipy.stats import sem
sem(ages)[0] # I use the [] notation because the result of the sem() function is an array, but I want to see its only entry

Step 5: Identify the critical value for the chosen confidence level

The Central Limit Theorem tells us that the mean of the sample is also distributed according to a Gaussian. The confidence interval (CI) that corresponds to a confidence level $\gamma$ of 95% tells us that 95% of the area of the Gaussian is contained between the CI values. While the CI of a Gaussian curve will depend on the mean and standard deviation of the specific Gaussian, we can use the properties of a normalized Gaussian centered on the zero (the z-distribution) and express the CI of our estimate in terms of a critical value, i.e., the corresponding interval for the z-distribution. In practice, $CI = m\pm z_\gamma \sigma_m$, where $\sigma_m$ is the SEM, and $z_\gamma$ is the critical value that corresponds to the given confidence level. Typical confidence levels and corresponding critical values are: 
* 90% -> $z_{90}=1.64$
* 95% -> $z_{95}=1.96$
* 99% -> $z_{99}=2.58$

SciPy also provides an easy way to compute critical values for arbitrary confidence levels.

In [ ]:
from scipy.stats import norm
confidence = 0.9999
norm.ppf((1+confidence)/2)

Step 6: Compute the error of your measure and fix the significant figures accordingly

In [ ]:
confidence = 0.95
error_ci95 = sem(ages)[0]*norm.ppf((1+confidence)/2)
print(error_ci95)

In [ ]:
print(f'The measured age is {mean_sample:4.1f} \u00B1 {error_ci95:3.1f} years')

## How to... Report Measurement Estimates and Confidence Intervals for Small Samples

When you have a small sample, say 3 measurements instead of 20, instead of using the z-distribution one needs to use the Student's t-distribution. Steps 1 to 4 of the section above stay the same, but Step 5 now requires to identify a specific critical value. The additional challenge is the fact that the t-distribution depends on one additional parameter, the degrees of freedom of the sample (for our applications $N-1$). While normally you would have to look up critical values of t statistics on ugly tables, SciPy allows us to compute them on the fly.

In [ ]:
from scipy.stats import t
confidence = 0.95
error_ci95 = sem(ages)[0]*t.ppf((1+confidence)/2,df=(len(ages)-1))
print(error_ci95)

In [ ]:
print(f'The measured age is {mean_sample:2.0f} \u00B1 {error_ci95:1.0f} years')

The confidence interval using t-distribution will always be larger than the one of a Gaussian, but the two will become more and more similar as the number of samples gets larger. In our example the difference is minimal, although it makes us report a bigger error because of rounding. Usually less than 10 samples will show a significant difference between the two approaches. 

## How to... Propagate Errors for Functions of One Variable

For the simpler case of a function of a single random variable, $y=f(x)$, one can compute the spread on the result as $\sigma_y = \left|\frac{df}{dx}\right|\sigma_x$. What is the origin of this formula?


In [ ]:
# @title Functions to load the data { display-mode: "form" }
import sympy as sp
x = sp.symbols('x')
function = 'x**2' # @param {type:"string"}
derivative = sp.diff(function,x)
n_measures=10000
mean_x = 2 # @param {type:"number"}
std_x = 0.4 # @param {type:"number"}
mu_x = mean_x
sigma_x = std_x
X = np.random.normal(mu_x,sigma_x,n_measures)
funct = sp.lambdify(x, function, "numpy")  
Y = funct(X)
dfunct = sp.lambdify(x, derivative, "numpy")
error_propagation = dfunct(X.mean())*X.std(ddof=1)
print("The spread on the result is {:0.3f}, while the error propagation formula gives us {:.3f}".format(Y.std(ddof=1),error_propagation))
# Create a Figure, which doesn't have to be square.
fig = plt.figure(constrained_layout=True)
# Create the main axes, leaving 25% of the figure space at the top and on the
# right to position marginals.
gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.05, hspace=0.05)
# Create the Axes.
ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)
# Draw the scatter plot and marginals.
ax.scatter(X,Y,alpha=0.1)
ax.set_xlabel('X')
ax.set_ylabel('Y')

ax_histx.tick_params(axis="x", labelbottom=False)
ax_histx.hist(X,bins=100)

ax_histy.tick_params(axis="y", labelleft=False)
ax_histy.hist(Y, bins=100, orientation='horizontal')

x=np.linspace(X.min(),X.max(),100)
y=funct(X.mean())+dfunct(X.mean())*(x-X.mean())
ax.plot(x,y,'red')

ax.annotate("",
            xy=(X.mean()-X.std(), funct(X.mean()-X.std())), xycoords='data',
            xytext=(X.mean()-X.std(), ax.get_ylim()[1]), textcoords='data',
            arrowprops=dict(arrowstyle="-", linestyle=":")
            )
ax.annotate("",
            xy=(X.mean()+X.std(), funct(X.mean()+X.std())), xycoords='data',
            xytext=(X.mean()+X.std(), ax.get_ylim()[1]), textcoords='data',
            arrowprops=dict(arrowstyle="-", linestyle=":")
            )
ax.annotate("",
            xy=(X.mean()-X.std(), funct(X.mean()-X.std())), xycoords='data',
            xytext=(ax.get_xlim()[1], funct(X.mean()-X.std())), textcoords='data',
            arrowprops=dict(arrowstyle="-", linestyle=":")
            )
ax.annotate("",
            xy=(X.mean()+X.std(), funct(X.mean()+X.std())), xycoords='data',
            xytext=(ax.get_xlim()[1], funct(X.mean()+X.std())), textcoords='data',
            arrowprops=dict(arrowstyle="-", linestyle=":")
            )

plt.show()

If you don't know how to compute the derivative of your function with respect to the random variable, you should: 
1. Refresh your Calculus (always a great idea)

or

2. Use SymPy, a symbolic python module that can compute the formula of the derivative for you

In [ ]:
x = sp.symbols('x')
function = 'x**2' # you can write the expression of your function as a string
derivative = sp.diff(function,x) # this gets the analytical expression of df/dx
print(derivative)

While the `function` and `derivative` above are just strings, `SymPy` can convert them to Numpy functions, so you can use them to propagate your errors

In [ ]:
x = sp.symbols('x')
function = 'x**2' 
derivative = sp.diff(function,x) 
mu_x = 2.
sigma_x = 0.4 
mu_y = sp.lambdify(x,function,'numpy')(mu_x)
sigma_y = sp.lambdify(x,derivative,'numpy')(mu_x) * sigma_x
print(f"The random variable x has a mean value of {mu_x} and a spread of {sigma_x}")
print(f"Given the expression y={function}, the mean of the result is {mu_y} and its associated spread is {sigma_y}")

## How to... Propagate Errors for Functions of Multiple Variables

What happens when we have a function that depends on multiple random variables? Given the expression $z=f(x,y,...)$, and assuming that the random variables are **uncorrelated**, the spread on the result is $\sigma_z^2 = \left|\frac{\partial f}{\partial x}\right|^2\sigma_x^2 + \left|\frac{\partial f}{\partial y}\right|^2\sigma_y^2 + ...$.  

For the simple case of the sum of two random variables, doing the math on the general formula reported above shows that the variance of the result is the sum of the variances of the random variables, i.e. $\sigma_{x+y}^2 = \sigma_x^2 + \sigma_y^2$. In fact, by generalizing this formula to the sum of $N$ random variables taken from the same distribution, we can easily get the expression of the standard error of the mean. 

If you don't know how to compute partial derivatives for functions of multiple variables: 
1. Refresh your upper-level Calculus 

or

2. Use `SymPy` to do the math for you

In [ ]:
x, y = sp.symbols(['x','y'])
function = 'x*y' 
partialf_dx = sp.diff(function,x) 
partialf_dy = sp.diff(function,y)
# first variable, mean and spread
mu_x = 0.2
sigma_x = 0.1
# second variable, mean and spread
mu_y = 3.
sigma_y = 1.
# result, mean and spread
mu_z = sp.lambdify([x,y],function,'numpy')(mu_x,mu_y)
sigma_z = np.sqrt( (sp.lambdify([x,y],partialf_dx,'numpy')(mu_x,mu_y))**2 * sigma_x**2 \
                  + (sp.lambdify([x,y],partialf_dy,'numpy')(mu_x,mu_y))**2 * sigma_y**2)
print(f"Given two random variables x = {mu_x} and y = {mu_y} and the expression z={function},\
      \nthe mean of the result is z={mu_z} and its associated spread is {sigma_z}")

It is instructive to check what happens if instead of summing two different variables, we sum the same variable twice, $z=x+x$. Is the result of the error propagation formula still consistent with the formula we derived in the previous section for a function of a single variable? Why or why not?

## How to... Perform Linear Regression wrt a Single Variable

If two variables are linked by a relationship, we can try to use one to predict the other. There are several different ways we can try to find and characterize relationships between two variables. Some approaches are very flexible and don't make any assumption on the type of relation, while other approaches (called parametric) introduce some kind of analytical relation and use the available data to fit its parameters. The simplest relation between two variables is a linear model: $Y=aX+b$. In general, in regression model we distinguish between independent variable(s) ($X_1$, $X_2$, $X_3$, ..., sometimes called features) and dependent variable ($Y$, sometime called label). The distinction is generally motivated by practical reasons, usually an accurate measure of $Y$ is much more complicated than measuring $X$, which is why we use the latter to predict the former. 

Given an analytical model, such as the linear relation above, the task of machine learning is to use the available data to fit the model, so that it can then be used to make predictions. These steps are exactly reflected in the strategy used by the machine-learning algorithms in the Scikit-Learn library, also known as `Sklearn`. In the following we will see how to use `Sklearn` to perform a linear regression.

Step 1: Import and check your data

In [ ]:
file='protein.csv'
data=pd.read_csv(path+file)
data.plot.scatter('SUR_CONT_10.0','VOL_CONT_5.0')
plt.show()

Step 2: Create an instance of a `LinearRegression` algorithm with default settings (no arguments)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression() # note that we can use any name we like on the left-hand side

The `lr` object created above is the one we will use to perform all the actions involved in the linear regresion. It will also store all the important quantities related to the model, in particular the values of the slope, $a$, and of the intercept, $b$, that best fit the data. However, at this stage the object has not seen the data yet.

Step 3: Create a `Numpy` array with the values of the independent variable, $X$, and another `Numpy` array with the corresponding values of the dependent variable, $Y$. 

In [ ]:
y=data['VOL_CONT_5.0'].values # the values attribute of the DataFrame extracts only the column values
x=data['SUR_CONT_10.0'].values.reshape(-1,1) # if we have a single independent variable X 
# we need to reshape it, because machine-learning algorithms are used to multidimensional features

Step 4: Feed the data to the algorithm, to calculate the parameters that best fit the data.

In [ ]:
lr.fit(x,y)

Once the model has been fit, you can see that it contains a few new attributes, which also include to the computed parameters

In [ ]:
#dir(lr)

In [ ]:
print(f"The fitted linear model has an intercept of {lr.intercept_:.4f} and a slope of {lr.coef_[0]:.4f}")

Step 5: Plot the original y vs. x and the values predicted by the linear model

Given that this is a simple linear model and we can access the intercept and the slope, we could just use those to calculate the equation of the fit. However, it is easier to just use the `predict()` method of the `lr` algorithm on a sensible range of values for the variable x

In [ ]:
x_predict = np.linspace(400,1000,10).reshape(-1,1) # we can choose the extrema that we want and how many points we like
y_predict = lr.predict(x_predict)
plt.scatter(x,y,s=3,label='data')
plt.plot(x_predict,y_predict,color='C1',label='linear regression')
plt.xlabel('SUR_CONT_10.0')
plt.ylabel('VOL_CONT_5.0')
plt.legend()
plt.show()

Step 6: Compute the quality of the fit, i.e., $R^2$ value, a.k.a. the coefficient of determination

In [ ]:
R2 = lr.score(x,y)
print(f"The R2 coefficient of the linear fit is {R2:.4f}")

This coefficient describes how much of the variation in the dependent variable is actually explained by the variation in the independent variable(s). For good fits the coefficient gets close to 1, while poor fits have coefficients close to 0. It can also be computed from the data in terms of the sum of squares $R^2=1-\frac{RSS}{TSS}$, where
* The Total Sum of Squares (TSS) is the sum of the square of the difference of each y value from the mean value of y
* The Residual Sum of Squares (RSS) is the sum of the square of the difference of each y value from the value predicted by the fit.

The TSS tells us how much variability is present in the y variable. The RSS tells us how much residual variability is present with respect to the linear regression fit. If the fit is good, all the variability in $y$ is explained by the model, so the RSS is very small compared to the TSS and $R^2$ is close to 1. On the contrary, if there is no linear variation (i.e. the model is flat around the average $y$) the two sum of squares are identical and $R^2$ goes to zero. 